## Project - Deflector

In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [2]:
#-- REAL WORLD configuration, which is attached to the physics engine
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/game_scene.g")

In [3]:
# Configure Physics Engine
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
# Attach Camera Sensor
S.addSensor("camera")
camera = RealWorld.frame("camera")

In [4]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/game_scene.g")

In [5]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [6]:
Xstart = C.getFrameState()
# C.selectJoints(["A_finger1", "A_finger2"], True)

In [7]:
q = S.get_q()
# q = C.getJointState()
print(q.shape)

(16,)


In [8]:
# [y,J] = C.evalFeature(ry.FS.oppose, ["A_finger1", "A_finger2", "ball1"])
# print(y.shape)
# print(J.shape)
# y = y * min(.008/np.linalg.norm(y), 1.)
# q.shape
# q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ y


In [9]:
# q = q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ y

In [10]:
tau = .01
t = 0

while True:
    t = t+1
    
    if t%5 == 0:
        [rgb, depth] = S.getImageAndDepth()
        
    if t==2000:
        break
        
    time.sleep(0.01)
    q = S.get_q()
    
    if t<=300:
        [y,J] = RealWorld.evalFeature(ry.FS.positionDiff, ["A_gripperCenter", "ball2"])
        vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y)
#     if t>=300 and S.getGripperIsGrasping("A_gripper"):
#         [y,J] = RealWorld.evalFeature(ry.FS.positionDiff, ["A_gripperCenter", "ball1"])
#         vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y)
    
        
#     if S.getGripperIsGrasping("A_gripper"):
#         [y,J] = C.evalFeature(ry.FS.position, ["A_gripper"]);
#         q = q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-2e-4]

#     if t==300:
#         S.closeGripper("A_gripper")
        
#     if t==900:
#         S.openGripper("A_gripper")
        
#     if S.getGripperIsGrasping("gripper"):
#         [y,J] = C.evalFeature(ry.FS.position, ["gripper"]);
#         q = q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-2e-4]
    
    S.step(vel, tau, ry.ControlMode.velocity)